# RESUMO DAS ABORDAGENS

Abordagem 1 (A1): remoção de outliers baseada na variável Valor Unitário Atualizado (R$/m2) e utilização deste atributo como variável dependente.

Abordagem 2 (A2): transformação da variável Área do Terreno com logaritmo
neperiano (ln), sem remover outliers e ln de Valor Total Atualizado como variável dependente.

Abordagem 3 (A3) Transformação do atributo Valor Unitário Atualizado com ln, remoção da variável Área do Terreno e ln de Valor Unitário Atualizado como
variável dependente.

## Instalação de Pacotes Necessários

In [ ]:
%time
import os
import numpy as np

!pip install --upgrade pandas
import pandas as pd
from pandas import read_csv

import math as math

## Importação de Dados Geolocalizados

In [ ]:
!pip install --upgrade gspread

In [ ]:
# autorização de acesso ao Google Drive  

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
planilha = gc.open('nome_basededados')
pagina = planilha.sheet1
pagina.row_values(1)

In [ ]:
df = pd.DataFrame(pagina.get_all_records())

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df = df.dropna()

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.dtypes

In [ ]:
df = df.astype({"% Superior Completo": float})
df = df.astype({"% Rede Geral de Distribuicao de Agua": float})
df = df.astype({"% Microcomputador com Acesso a Internet": float})
df = df.astype({"Rendimento Domiciliar Medio": float})

df.dtypes

In [ ]:
df.describe()

In [ ]:
df[df["Fonte"]  == "ME"].count()

In [ ]:
df[df["Fonte"]  == "EB"].count()

In [ ]:
df.drop([2730,4341,4581], axis=0, inplace=True)

# Abordagem 1

Remoção de _outliers_ e _Valor Unitário Atualizado_ como variável dependente

In [ ]:
# Método de Tukey (algoritmo para tratamento de outliers)

def tukeys_method(df, variable):
    # Recebe dois parâmetros: dataframe & variável de interesse como string
    q1 = df[variable].quantile(0.25)
    q3 = df[variable].quantile(0.75)
    iqr = q3-q1
    inner_fence = 1.5*iqr
    outer_fence = 3*iqr
    
    # limites internos
    inner_fence_le = q1-inner_fence
    inner_fence_ue = q3+inner_fence
    
    # limites externos
    outer_fence_le = q1-outer_fence
    outer_fence_ue = q3+outer_fence
    
    outliers_prob = []
    outliers_poss = []
    for index, x in enumerate(df[variable]):
        if x <= outer_fence_le or x >= outer_fence_ue:
            outliers_prob.append(index)
    for index, x in enumerate(df[variable]):
        if x <= inner_fence_le or x >= inner_fence_ue:
            outliers_poss.append(index)
    return outliers_prob, outliers_poss

In [ ]:
df.columns

In [ ]:
outliers_prob, outliers_poss = tukeys_method(df,'ValorUnitarioAtualizado')

In [ ]:
outliers_poss

In [ ]:
len(outliers_poss)

In [ ]:
df_limpo = df.drop(outliers_poss)

In [ ]:
df_limpo.describe()

In [ ]:
X_columns = ['CapitalUF', 'IDHM_2010', 'Terreno', 'IdadeAparente',
       'Rendimento Domiciliar Medio', 'qtd_equipamento',
       'qtd_shopping', 'qtd_estabelecimento', 'CUB*AreaConstruida', 'Vocacao_Comercial', 'Vocacao_Institucional',
       'Vocacao_Misto', 'Vocacao_Residencial', 'AreaProjetadaTratada']

X = df_limpo[X_columns]

y = df_limpo['ValorUnitarioAtualizado']
y = y[:,np.newaxis]

# Abordagem 2

Transformação da variável _Área do Terreno_ com logaritmo neperiano (ln), sem remover _outliers_, e ln de _Valor Total Atualizado_ como variável
dependente.

In [ ]:
df['LnValorTotalAtualizado'] = np.log(df['ValorTotalAtualizado'])

In [ ]:
df['LnAreaProjetadaTratada'] = np.log(df['AreaProjetadaTratada'])

In [ ]:
X_columns = ['CapitalUF', 'IDHM_2010', 'Terreno', 'IdadeAparente',
       'Rendimento Domiciliar Medio', 'qtd_equipamento',
       'qtd_shopping', 'qtd_estabelecimento', 'CUB*AreaConstruida', 'Vocacao_Comercial', 'Vocacao_Institucional',
       'Vocacao_Misto', 'Vocacao_Residencial', 'LnAreaProjetadaTratada']

X = df[X_columns]

y = np.log(df['ValorTotalAtualizado'])
y = y[:,np.newaxis]

# Abordagem 3

Transformação do atributo _Valor Unitário Atualizado_ com
ln, remoção da variável _Área do Terreno_ e ln de _Valor Unitário Atualizado_ como variável dependente.

In [ ]:
X_columns = ['CapitalUF', 'IDHM_2010', 'Terreno', 'IdadeAparente',
       'Rendimento Domiciliar Medio', 'qtd_equipamento',
       'qtd_shopping', 'qtd_estabelecimento', 'CUB*AreaConstruida', 'Vocacao_Comercial', 'Vocacao_Institucional',
       'Vocacao_Misto', 'Vocacao_Residencial']

X = df[X_columns]

y = np.log(df['ValorUnitarioAtualizado'])
y = y[:,np.newaxis]